In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as  mpl

In [2]:
mpl.rcParams[u'font.sans-serif'] = ['simhei']
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
def readData(y,s,c):
    df = pd.read_csv("lvr_land/"+ y +"-s" + s + "/" + c + "_lvr_land_A.csv",encoding='utf-8',low_memory=False)
    return df

In [4]:
def combineDF(df1,df2):
    df = pd.concat([df1,df2],axis = 0)
    return df

In [5]:
def selectFarmland(df):
    
    df = df[["鄉鎮市區","土地移轉總面積(平方公尺)","交易標的","都市土地使用分區","非都市土地使用分區","非都市土地使用編定","總價(元)","單價(元/平方公尺)"]]
    df = df[(df.交易標的 == "土地")]  
    df_farmland = df[(((df.非都市土地使用分區 == '特定農業區')|(df.非都市土地使用分區 == "一般農業區"))&(df.非都市土地使用編定 =="農牧用地"))]
    #排除都市土地
    df_farmland = df_farmland[df_farmland["都市土地使用分區"].isnull()]
    return df_farmland

In [6]:
def fillNanprice(df_farmland):
    #modify zero or nan price
    df_ok = df_farmland[(df_farmland["單價(元/平方公尺)"].notnull())&(df_farmland["單價(元/平方公尺)"] != "0")]
    df = df_farmland[(df_farmland["單價(元/平方公尺)"].isnull())|(df_farmland["單價(元/平方公尺)"] == "0")]
    df = df[(df['土地移轉總面積(平方公尺)'] != "0")&(df['總價(元)'] != "0")]
    df["單價(元/平方公尺)"] = calculateprice(df['總價(元)'],df['土地移轉總面積(平方公尺)'])
    df_farmland = pd.concat([df_ok,df],axis = 0)
    return df_farmland

In [7]:
def calculateprice(total_price,area):
    return pd.to_numeric(total_price)/pd.to_numeric(area)

In [8]:
def calculateMean(df_farmland):

    landpriceMean = pd.to_numeric(df_farmland['單價(元/平方公尺)']).mean()
   
    return landpriceMean

In [9]:
def calculateGain(cityMeandict,div):
    
    #dict to df
    arr = list(cityMeandict.items())
    df = pd.DataFrame(arr, columns=['年份','單價'])
    
    #  #逐年漲幅
    gain = df["單價"].pct_change(div)
    
    return gain

In [10]:
def grouptown_calculateMean(df,list_all_town):
    
    grouped=pd.to_numeric(df['單價(元/平方公尺)']).groupby(df['鄉鎮市區'])
    town_dict = {}
    for key,group in grouped:  
        m = grouped.mean()
        town_dict[key]= m[key]*0.3025 # convert m^2 to ping
        if key not in list_all_town:
            list_all_town.append(key)
    return {"towndict":town_dict,"town_list":list_all_town}

In [11]:
def plottown_average(towndict,list_all_town):
    list_color = ['#393b79','#5254a3','#6b6ecf','#9c9ede','#637939','#8ca252','#b5cf6b','#cedb9c','#8c6d31','#bd9e39','#e7ba52','#e7cb94','#843c39','#ad494a','#d6616b','#e7969c','#7b4173','#a55194','#ce6dbd','#de9ed6','#3182bd','#6baed6','#9ecae1','#c6dbef','#e6550d','#fd8d3c','#fdae6b','#fdd0a2','#e6550d','#fd8d3c','#fdae6b','#fdd0a2','#636363','#969696','#bdbdbd','#d9d9d9']
    color = 0
    for town in list_all_town: 
        town_year_dict = {}
        for key in towndict:   
            if town in towndict[key]:
                town_year_dict[key] = towndict[key][town]
        
        plt.plot(town_year_dict.keys(),town_year_dict.values(),marker='o',linestyle='solid',label=town if(town!='fa72埔鄉')else'鹽埔鄉')
        
        plt.xticks(range(len(towndict)), towndict.keys(),rotation=90) 
        plt.xlabel("年")
        plt.ylabel("(元/平方公尺)")
        plt.legend(loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
        plt.subplots_adjust(right=0.7)
        plt.title(list_city[i] + " "+"各鄉鎮市區"+"單價")
        plt.grid()
        plt.savefig("town-"+list_city[i])
        plt.show()
        color = color + 1
        
        for i in range(101,108):
            print("|"+str(i)+"|",end='')
            if np.isnan(town_year_dict[i]) == False:
                print(str(town_year_dict[i])+"|") 
            else:
                print("-|")

In [12]:
list_cityfilename = ['A','B','C','D','E','F','G','H','J','K','M','N','P','Q','T','U','V']
list_city = ['台北市','台中市','基隆市','台南市','高雄市','新北市','宜蘭縣','桃園縣','新竹縣','苗栗縣','南投縣','彰化縣','雲林縣','嘉義縣','屏東縣','花蓮縣','台東縣']

#list_city  = ['花蓮縣','苗栗縣','新竹縣','台東縣','屏東縣',]
#list_cityfilename = ['U','K','J','V','T']
#combine嘉義縣市(Q,I) 新竹縣市(J,O)

i=0
taiwan = {} #各縣市逐年平均
city_gain = {} #縣市逐年漲幅
全國縣市單價 = {}
for c in list_cityfilename: #every city
    
    s = 4 # from 101-s4 to 107-s2
    cityMeandict = {} #各城市逐年平均
    townMeandict = {} #各鄉鎮逐年平均
    list_all_town = [] #該城市的鄉鎮列表
    for y in range(101,108): #every year
        df = pd.DataFrame()
        
        while s <= 4:
            # read data
            df_s =  readData(str(y),str(s),c)
            
            if c=='Q':# combine嘉義縣市(Q,I) 
                df_s = combineDF(df_s,readData(str(y),str(s),'I'))
            if c=='J':# combine新竹縣市(J,O)   
                df_s = combineDF(df_s,readData(str(y),str(s),'O'))
               
            df = combineDF(df,df_s)
            
            if y == 107 and s == 2:
                break
            s = s+1
        s=1
        # select farmland
        df_farmland = selectFarmland(df)
        # fill nan price
        df_farmland = fillNanprice(df_farmland)
        
        # 縣市年的平均
        mean = calculateMean(df_farmland)
        cityMeandict[str(y)] = mean*0.3025# convert m^2 to ping
       
        # 鄉鎮
        result = grouptown_calculateMean(df_farmland,list_all_town)
        townMeandict[str(y)] = result["towndict"]
    全國縣市單價[list_city[i]] = cityMeandict
    
    print(list_city[i])
    #各鄉鎮逐年單價排名
    print("各鄉鎮逐年單價排名:")
    table_dict = {}
    for y in range(101,108):
        sorted_town = sorted(townMeandict[str(y)].items(),key = lambda s:s[1],reverse=True)
        table_dict[y] = sorted_town
   
    print("|排名\年",end='')
    for y in range(101,108):
        print("|"+str(y),end='')
    print("|")
    for y in range(101,108):
        print("|------",end='')
    print("|")
    for t in range(len(list_all_town)):
        print("|"+str(t+1)+"|",end='')
        for y in range(101,108):
            if t < len(table_dict[y]):
                print(table_dict[y][t][0]+" / "+"{:.1f}".format(table_dict[y][t][1])+"|",end="") 
            else:
                print("-|",end="")
        print("")
        
    #plottown_average(townMeandict,list_all_town)
    
#     #鄉鎮逐年漲幅
#     print("鄉鎮逐年漲幅")
#     鄉鎮逐年平均單價={}
#     鄉鎮歷年平均單價 = {}
#     鄉鎮年度漲幅 = {}
#     鄉鎮逐年漲幅 = {}
#     for t in list_all_town:
#         鄉鎮逐年平均單價 = {}
#         for y in range(101,108):
#             if t in townMeandict[str(y)].keys():
#                 鄉鎮逐年平均單價[y] = townMeandict[str(y)][t]
#         鄉鎮歷年平均單價[t] = 鄉鎮逐年平均單價
      
#         gain = calculateGain(鄉鎮歷年平均單價[t],1)
#         year_list = list(鄉鎮歷年平均單價[t].keys())
        
#         y=0
#         鄉鎮逐年漲幅 = {}
#         for g in gain:
#             鄉鎮逐年漲幅[year_list[y]] = g
#             y+=1
#         鄉鎮年度漲幅[t] = 鄉鎮逐年漲幅
    
#     for_sort = {}
#     各年漲幅 = {}
#     sorted_gain = {}
#     for y in range(102,108):
#         for_sort = {}
#         for t in list_all_town:
#             if y in 鄉鎮年度漲幅[t].keys():
#                 for_sort[t] = 鄉鎮年度漲幅[t][y]
#         各年漲幅[y] = for_sort

#     for y in range(102,108):
#         sorted_gain[y] = sorted(各年漲幅[y].items(),key=lambda s:s[1],reverse=True)
    
#     sorted_dict = {}
#     for y in range(102,108):
#         sorted_dict[y] = dict(sorted_gain[y])
        
#     for s in list_all_town:
#         print("|"+str(s),end="")
#     print("|")
#     for s in list_all_town:
#         print("|------",end="")
#     print("|")
    
#     for y in range(102,108):
#         print("|"+str(y)+"|",end='')
#         for t in list_all_town:
#             if t in sorted_dict[y].keys():
#                 print("{:.1f}".format(sorted_dict[y][t]*100)+"|",end='')  
#             else:
#                 print("-|",end='')
#         print("")
        
    #鄉鎮總漲幅
    print("鄉鎮總漲幅:")
    鄉鎮逐年平均單價={}
    鄉鎮歷年平均單價 = {}
    鄉鎮年度漲幅 = {}
    鄉鎮逐年漲幅 = {}
    for t in list_all_town:
        鄉鎮逐年平均單價 = {}
        for y in range(101,108):
            if t in townMeandict[str(y)].keys():
                鄉鎮逐年平均單價[y] = townMeandict[str(y)][t]
        鄉鎮歷年平均單價[t] = 鄉鎮逐年平均單價
      
        gain = calculateGain(鄉鎮歷年平均單價[t],len(鄉鎮歷年平均單價[t].keys())-1)
        for g in gain:
            if np.isnan(g) == False:
                 鄉鎮年度漲幅[t] = g
    
    sorted_gain = sorted(鄉鎮年度漲幅.items(),key=lambda s:s[1],reverse=True)

    print("|排名\年|107|")

    print("|------|------|")
    for h in range(len(鄉鎮年度漲幅.keys())):
        print("|"+str(h)+"|",end="")  
        print(sorted_gain[h][0]+"/"+"{:.1f}".format(sorted_gain[h][1]*100)+"|")     
      
    i = i+1
    

台北市
各鄉鎮逐年單價排名:
|排名\年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|
鄉鎮總漲幅:
|排名\年|107|
|------|------|
台中市
各鄉鎮逐年單價排名:
|排名\年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|
|1|大雅區 / 5302.7|大里區 / 6013.6|大里區 / 6596.9|潭子區 / 8894.3|豐原區 / 5669.4|潭子區 / 6237.3|潭子區 / 15125.0|
|2|太平區 / 4513.0|大雅區 / 5357.5|大雅區 / 6194.5|大里區 / 7053.6|潭子區 / 5593.2|大里區 / 5513.5|大里區 / 5647.8|
|3|大里區 / 4340.8|烏日區 / 4878.9|龍井區 / 6013.4|大雅區 / 5976.5|太平區 / 5512.4|豐原區 / 5248.2|太平區 / 4754.7|
|4|烏日區 / 3648.2|龍井區 / 4622.5|潭子區 / 5149.1|龍井區 / 5193.2|大里區 / 4994.7|龍井區 / 5049.5|豐原區 / 4674.3|
|5|沙鹿區 / 3280.9|太平區 / 4124.3|太平區 / 4853.5|烏日區 / 4725.4|龍井區 / 4978.2|大雅區 / 4391.7|龍井區 / 4110.2|
|6|霧峰區 / 2661.4|豐原區 / 4043.2|烏日區 / 4589.3|霧峰區 / 4142.4|烏日區 / 4598.6|烏日區 / 3993.3|神岡區 / 3719.6|
|7|新社區 / 2639.0|霧峰區 / 3531.3|霧峰區 / 4339.4|豐原區 / 4103.8|大雅區 / 4551.9|霧峰區 / 3984.5|大雅區 / 3620.0|
|8|神岡區 / 2011.3|沙鹿區 / 3248.1|豐原區 / 4157.2|太平區 / 3830.1|霧峰區 / 3699.2|太平區 / 3359.9|大肚區 / 3611.3|
|9|

宜蘭縣
各鄉鎮逐年單價排名:
|排名\年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|
|1|羅東鎮 / 3634.1|羅東鎮 / 1974.6|羅東鎮 / 2750.3|羅東鎮 / 2584.5|礁溪鄉 / 2108.8|羅東鎮 / 2700.7|羅東鎮 / 2836.1|
|2|頭城鎮 / 1463.9|宜蘭市 / 1794.6|宜蘭市 / 1902.1|礁溪鄉 / 1778.8|宜蘭市 / 1986.4|宜蘭市 / 1766.8|礁溪鄉 / 2375.3|
|3|五結鄉 / 1403.5|礁溪鄉 / 1296.8|礁溪鄉 / 1355.3|宜蘭市 / 1767.2|羅東鎮 / 1867.2|礁溪鄉 / 1649.6|宜蘭市 / 1760.8|
|4|宜蘭市 / 1372.6|員山鄉 / 1235.3|員山鄉 / 1327.4|五結鄉 / 1496.7|頭城鎮 / 1461.7|五結鄉 / 1414.1|員山鄉 / 1228.0|
|5|礁溪鄉 / 1186.1|五結鄉 / 1227.0|頭城鎮 / 1288.8|員山鄉 / 1419.8|壯圍鄉 / 1257.9|員山鄉 / 1269.0|五結鄉 / 1152.7|
|6|員山鄉 / 978.6|冬山鄉 / 989.4|五結鄉 / 1201.2|頭城鎮 / 1330.7|五結鄉 / 1234.4|頭城鎮 / 1155.4|蘇澳鎮 / 1124.7|
|7|壯圍鄉 / 869.8|頭城鎮 / 863.5|冬山鄉 / 1160.1|壯圍鄉 / 1285.4|冬山鄉 / 1232.1|冬山鄉 / 974.3|頭城鎮 / 1041.1|
|8|冬山鄉 / 831.6|壯圍鄉 / 796.4|三星鄉 / 1065.0|冬山鄉 / 1274.5|員山鄉 / 1174.2|三星鄉 / 939.7|冬山鄉 / 887.5|
|9|三星鄉 / 796.1|三星鄉 / 778.7|壯圍鄉 / 1018.5|三星鄉 / 1146.1|三星鄉 / 1124.2|壯圍鄉 / 877.0|壯圍鄉 / 828.6|
|10|蘇澳鎮 / 448.3|蘇澳鎮 / 748.2|蘇澳鎮 / 743.3|蘇澳鎮 / 960.0|蘇澳鎮 /

雲林縣
各鄉鎮逐年單價排名:
|排名\年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|
|1|虎尾鎮 / 653.4|虎尾鎮 / 991.2|虎尾鎮 / 1263.9|虎尾鎮 / 916.8|斗六市 / 936.7|虎尾鎮 / 883.4|虎尾鎮 / 981.5|
|2|斗六市 / 598.8|斗六市 / 751.7|斗六市 / 776.6|斗六市 / 855.5|虎尾鎮 / 888.7|西螺鎮 / 839.5|西螺鎮 / 844.7|
|3|斗南鎮 / 461.2|西螺鎮 / 689.6|西螺鎮 / 683.6|西螺鎮 / 656.3|西螺鎮 / 866.1|斗六市 / 733.4|斗六市 / 730.5|
|4|西螺鎮 / 429.5|斗南鎮 / 469.3|斗南鎮 / 452.7|莿桐鄉 / 491.9|麥寮鄉 / 508.5|莿桐鄉 / 567.3|二崙鄉 / 682.6|
|5|古坑鄉 / 412.7|莿桐鄉 / 421.4|麥寮鄉 / 450.6|麥寮鄉 / 489.0|莿桐鄉 / 502.0|土庫鎮 / 530.0|莿桐鄉 / 541.8|
|6|北港鎮 / 362.4|麥寮鄉 / 390.7|莿桐鄉 / 440.8|斗南鎮 / 468.5|土庫鎮 / 460.4|斗南鎮 / 466.8|土庫鎮 / 466.6|
|7|二崙鄉 / 351.7|土庫鎮 / 359.9|土庫鎮 / 411.6|土庫鎮 / 414.8|斗南鎮 / 430.6|二崙鄉 / 465.9|麥寮鄉 / 450.5|
|8|土庫鎮 / 347.8|二崙鄉 / 352.9|二崙鄉 / 373.8|林內鄉 / 403.8|二崙鄉 / 418.4|林內鄉 / 412.5|大埤鄉 / 409.8|
|9|林內鄉 / 336.3|崙背鄉 / 340.4|古坑鄉 / 347.5|二崙鄉 / 402.1|林內鄉 / 380.8|褒忠鄉 / 410.7|台西鄉 / 405.0|
|10|莿桐鄉 / 322.8|古坑鄉 / 332.9|大埤鄉 / 338.6|口湖鄉 / 399.7|古坑鄉 / 379.2|古坑鄉 / 403.7|斗南鎮 / 392.8|
|11|大埤鄉 / 308.1|褒

In [13]:
#縣市總漲幅
print("縣市總漲幅:")
縣市逐年漲幅={}
for t in list_city:
    gain = calculateGain(全國縣市單價[t],6)#總漲幅 argu2 = 6
    y=101
    年度漲幅 = {}
    for g in gain:
        年度漲幅[y] = g
        y=y+1
    縣市逐年漲幅[t] = 年度漲幅
for_sort = {}
各年漲幅 = {}
sorted_gain = {}
for y in range(102,108):
    for_sort = {}
    for t in list_city:
        for_sort[t] = 縣市逐年漲幅[t][y]
    各年漲幅[y] = for_sort

for y in range(102,108):
    sorted_gain[y] = sorted(各年漲幅[y].items(),key=lambda s:s[1],reverse=True)

print("|排名\年|漲幅|")

print("|------|------|")

for i in range(0,17):
    if np.isnan(sorted_gain[y][i][1]) == False:
        print("|"+str(i-1),end="")
        print("|"+sorted_gain[y][i][0]+"/"+ "{:.1f}".format(sorted_gain[y][i][1]*100)+"|")


縣市總漲幅:
|排名\年|漲幅|
|------|------|
|1|新北市/110.0|
|2|台東縣/100.4|
|3|苗栗縣/56.8|
|4|高雄市/44.5|
|5|屏東縣/44.4|
|6|彰化縣/33.2|
|7|雲林縣/28.2|
|8|台南市/22.5|
|9|花蓮縣/21.9|
|10|宜蘭縣/18.8|
|11|台中市/17.4|
|12|南投縣/13.4|
|13|桃園縣/10.3|
|14|嘉義縣/9.0|
|15|新竹縣/-24.6|


In [14]:
print("各縣市單價")
for i in list_city:
    print(i)
#     plt.plot(全國縣市單價[i].keys(),全國縣市單價[i].values(),marker='o',linestyle='solid')    
#     plt.xticks(range(len(全國縣市單價[i])), 全國縣市單價[i].keys(),rotation=90) 
#     plt.xlabel("年")
#     plt.ylabel("(元/坪)")
#     plt.title(i + "101-107平均單價")
#     plt.grid()
#     #plt.savefig(i + "101-107平均單價")
#     plt.show()
    
    print("|年",end='')
    for y in range(101,108):
        print("|"+str(y),end='')
    print("|")
    for y in range(101,109):
        print("|------",end='')
    print("|")
    print("|元/坪",end='')
    for h in range(101,108):
        print("|"+"{:.1f}".format(全國縣市單價[i][str(h)]),end='')
    print("|")

各縣市單價
台北市
|年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|------|
|元/坪|nan|nan|nan|nan|nan|nan|nan|
台中市
|年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|------|
|元/坪|2475.5|2961.3|2874.6|2750.5|2763.5|3028.7|2906.0|
基隆市
|年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|------|
|元/坪|nan|nan|nan|nan|nan|nan|nan|
台南市
|年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|------|
|元/坪|576.7|588.5|636.6|651.9|667.0|673.6|706.3|
高雄市
|年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|------|
|元/坪|722.5|874.7|964.8|1038.8|984.5|1103.8|1043.7|
新北市
|年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|------|
|元/坪|2295.0|3552.4|5579.6|4707.0|4176.3|3648.9|4819.6|
宜蘭縣
|年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|------|
|元/坪|1038.5|1045.5|1204.9|1386.6|1364.6|1205.5|1233.6|
桃園縣


In [16]:
#縣市逐年排名
print("縣市逐年排名")
average_dict = {}
for y in range(101,108): 
    average_rank = []
    for c in list_city:      
        average_rank.append((c,全國縣市單價[c][str(y)]))
    average_rank = sorted(average_rank, key=lambda s:s[1], reverse=True)
    average_dict[y] = average_rank

print("|排名\年",end='')
for i in range(101,108):
    print("|"+str(i),end='')
print("|")
for i in range(101,109):
    print("|------",end='')
print("|")
for i in range(len(list_city)):
    if np.isnan(average_dict[y][i][1]) == False:
        print("|"+str(i-1),end='')
        for y in range(101,108):
            print("|"+average_dict[y][i][0]+" / "+"{:.1f}".format(average_dict[y][i][1]),end='')
        print("|")
    

縣市逐年排名
|排名\年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|------|
|1|新竹縣 / 3245.1|新北市 / 3552.4|新北市 / 5579.6|新北市 / 4707.0|新北市 / 4176.3|新北市 / 3648.9|新北市 / 4819.6|
|2|台中市 / 2475.5|桃園縣 / 3093.0|桃園縣 / 3327.0|新竹縣 / 3319.7|新竹縣 / 3444.2|台中市 / 3028.7|台中市 / 2906.0|
|3|新北市 / 2295.0|台中市 / 2961.3|新竹縣 / 3180.1|桃園縣 / 2919.6|桃園縣 / 2849.6|新竹縣 / 2516.9|新竹縣 / 2448.2|
|4|桃園縣 / 2199.7|新竹縣 / 2478.6|台中市 / 2874.6|台中市 / 2750.5|台中市 / 2763.5|桃園縣 / 2450.3|桃園縣 / 2427.3|
|5|南投縣 / 1197.2|南投縣 / 1167.9|南投縣 / 1270.9|宜蘭縣 / 1386.6|宜蘭縣 / 1364.6|宜蘭縣 / 1205.5|南投縣 / 1358.2|
|6|宜蘭縣 / 1038.5|宜蘭縣 / 1045.5|宜蘭縣 / 1204.9|南投縣 / 1238.0|南投縣 / 1220.7|南投縣 / 1175.0|宜蘭縣 / 1233.6|
|7|彰化縣 / 832.2|苗栗縣 / 1026.1|苗栗縣 / 1124.9|彰化縣 / 1116.1|彰化縣 / 1121.5|彰化縣 / 1172.4|苗栗縣 / 1145.0|
|8|苗栗縣 / 730.4|彰化縣 / 1018.2|彰化縣 / 1080.0|高雄市 / 1038.8|苗栗縣 / 1090.9|高雄市 / 1103.8|彰化縣 / 1108.8|
|9|高雄市 / 722.5|高雄市 / 874.7|高雄市 / 964.8|苗栗縣 / 1021.4|高雄市 / 984.5|苗栗縣 / 967.3|高雄市 / 1043.7|
|10|台南市 / 576.7|台南市 / 588.5|台南市 / 636.6|花蓮縣 / 752.2|